# Imports

In [1]:
import json
import random

import ipywidgets as widgets
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image as IpImage
from IPython.display import display
from ipywidgets import Button, HBox, VBox
from PIL import Image as PilImage
from siuba import _, filter, group_by, summarize
from sklearn.cluster import KMeans

# %matplotlib widget

# Wrangle dataframe

## Load source dataframe

In [2]:
df = (
    pd.read_csv(
        "../data_in/draw-rectangles-around-the-gemma-cups-classifications.csv"
    )
    .sort_values(["user_name", "created_at", "subject_ids"])
    .reset_index()
)
df.head()

,index,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,1593,308331955,348850140,2227141.0,29c1c24442f5209f1de1,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 22:48:36 UTC,NaN,NaN,"{""source"":""api"",""session"":""62548b811c41e9bfb2d...","[{""task"":""T0"",""task_label"":""Draw a rectangle a...","{""54059293"":{""retired"":{""id"":75025956,""workflo...",54059293
1,1383,308318834,AfuoO.,2233913.0,7a63604d290de3070d92,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:44:41 UTC,NaN,NaN,"{""source"":""api"",""session"":""95c4812615321d1301c...","[{""task"":""T0"",""task_label"":""Draw a rectangle a...","{""54059285"":{""retired"":{""id"":75025998,""workflo...",54059285
2,1401,308319146,AfuoO.,2233913.0,4a1d04c27cf88ee6848b,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:46:12 UTC,NaN,NaN,"{""source"":""api"",""session"":""95c4812615321d1301c...","[{""task"":""T0"",""task_label"":""Draw a rectangle a...","{""54059293"":{""retired"":{""id"":75025956,""workflo...",54059293
3,1449,308319833,AfuoO.,2233913.0,e6058d710b12e4416d16,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:49:36 UTC,NaN,NaN,"{""source"":""api"",""session"":""95c4812615321d1301c...","[{""task"":""T0"",""task_label"":""Draw a rectangle a...","{""54059274"":{""retired"":{""id"":75025965,""workflo...",54059274
4,1462,308320070,AfuoO.,2233913.0,e9b266499b6e6731d12b,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:50:43 UTC,NaN,NaN,"{""source"":""api"",""session"":""95c4812615321d1301c...","[{""task"":""T0"",""task_label"":""Draw a rectangle a...","{""54059267"":{""retired"":{""id"":75026021,""workflo...",54059267


In [3]:
df.columns

Index(['index', 'classification_id', 'user_name', 'user_id', 'user_ip',
       'workflow_id', 'workflow_name', 'workflow_version', 'created_at',
       'gold_standard', 'expert', 'metadata', 'annotations', 'subject_data',
       'subject_ids'],
      dtype='object')

## Expand columns

### Expand metadata

In [4]:
metadata = df["metadata"].apply(lambda x: json.loads(x)).apply(pd.Series)
df_xµd = pd.concat(
    [
        df.drop(["metadata"], axis=1),
        metadata["viewport"].apply(pd.Series),
        metadata["interventions"].apply(pd.Series),
        metadata["subject_dimensions"].apply(pd.Series),
        metadata["subject_selection_state"].apply(pd.Series),
        metadata.drop(
            [
                "viewport",
                "interventions",
                "subject_dimensions",
                "subject_selection_state",
            ],
            axis=1,
        ),
    ],
    axis=1,
)
df_xµd = pd.concat(
    [
        df_xµd.drop([0], axis=1),
        df_xµd[0].apply(pd.Series),
    ],
    axis=1,
)
df_xµd.head(3)

,index,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,...,finished_at,live_project,user_language,user_group_ids,workflow_translation_id,seen_before,clientWidth,clientHeight,naturalWidth,naturalHeight
0,1593,308331955,348850140,2227141.0,29c1c24442f5209f1de1,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 22:48:36 UTC,NaN,...,2021-02-09T22:48:36.970Z,True,en,[],40817,NaN,870.0,870.0,1600.0,1600.0
1,1383,308318834,AfuoO.,2233913.0,7a63604d290de3070d92,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:44:41 UTC,NaN,...,2021-02-09T21:44:40.461Z,True,en,[],40817,NaN,475.0,475.0,1600.0,1600.0
2,1401,308319146,AfuoO.,2233913.0,4a1d04c27cf88ee6848b,17178,Draw rectangles around the gemma cups,5.8,2021-02-09 21:46:12 UTC,NaN,...,2021-02-09T21:46:12.238Z,True,en,[],40817,NaN,475.0,475.0,1600.0,1600.0


### Avoid same name column issues

In [5]:
df_xµd["retired_bool"] = df_xµd.retired
df_xµd = df_xµd.drop(["retired"], axis=1)
df_xµd["creation_time"] = df_xµd.created_at
df_xµd = df_xµd.drop(["created_at"], axis=1)
df_xµd

,index,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,gold_standard,expert,...,user_language,user_group_ids,workflow_translation_id,seen_before,clientWidth,clientHeight,naturalWidth,naturalHeight,retired_bool,creation_time
0,1593,308331955,348850140,2227141.0,29c1c24442f5209f1de1,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,870.0,870.0,1600.0,1600.0,True,2021-02-09 22:48:36 UTC
1,1383,308318834,AfuoO.,2233913.0,7a63604d290de3070d92,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,475.0,475.0,1600.0,1600.0,True,2021-02-09 21:44:41 UTC
2,1401,308319146,AfuoO.,2233913.0,4a1d04c27cf88ee6848b,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,475.0,475.0,1600.0,1600.0,True,2021-02-09 21:46:12 UTC
3,1449,308319833,AfuoO.,2233913.0,e6058d710b12e4416d16,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,475.0,475.0,1600.0,1600.0,True,2021-02-09 21:49:36 UTC
4,1462,308320070,AfuoO.,2233913.0,e9b266499b6e6731d12b,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,475.0,475.0,1600.0,1600.0,True,2021-02-09 21:50:43 UTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,2000,308517469,tat_qa,1994640.0,4d2d892606ac051e67d6,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,843.0,843.0,1600.0,1600.0,True,2021-02-10 21:31:46 UTC
2114,2001,308517552,tat_qa,1994640.0,4d2d892606ac051e67d6,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,en,[],40817,NaN,843.0,843.0,1600.0,1600.0,True,2021-02-10 21:32:09 UTC
2115,237,295508887,thadguidry,1662232.0,1f82beac278d06e47995,17178,Draw rectangles around the gemma cups,5.6,NaN,NaN,...,en,[],40817,NaN,869.0,869.0,1520.0,1520.0,False,2020-12-17 14:16:14 UTC
2116,238,295509285,thadguidry,1662232.0,1f82beac278d06e47995,17178,Draw rectangles around the gemma cups,5.6,NaN,NaN,...,en,[],40817,NaN,869.0,869.0,1520.0,1520.0,False,2020-12-17 14:18:06 UTC


### Split subject_data and annotations

In [6]:
dfx = pd.concat(
    [
        df_xµd.drop(["subject_data", "annotations"], axis=1),
        df_xµd["subject_data"]
        .apply(lambda x: list(json.loads(x).values())[0])
        .apply(pd.Series),
        df_xµd["annotations"].str[1:-1].apply(lambda x: json.loads(x)).apply(pd.Series),
    ],
    axis=1,
)
dfx = (
    pd.concat(
        [
            dfx.drop(["retired"], axis=1),
            dfx["retired"].apply(pd.Series),
        ],
        axis=1,
    )
    .rename(str.lower, axis="columns")
    .sort_values(by=["user_name", "filename", "created_at"])
)
dfx.head(3)

,index,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,gold_standard,expert,...,task_label,value,id,workflow_id,classifications_count,created_at,updated_at,retired_at,subject_id,retirement_reason
0,1593,308331955,348850140,2227141.0,29c1c24442f5209f1de1,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,Draw a rectangle around every visible gemma cup.,"[{'x': 696.6875610351562, 'y': 667.35845947265...",75025956.0,17178.0,19.0,2020-12-21T08:03:45.760Z,2021-02-09T21:20:31.286Z,2021-02-09T21:20:31.279Z,54059293.0,classification_count
4,1462,308320070,AfuoO.,2233913.0,e9b266499b6e6731d12b,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,Draw a rectangle around every visible gemma cup.,[],75026021.0,17178.0,29.0,2020-12-21T08:06:00.356Z,2021-02-09T21:26:30.248Z,2021-02-09T21:26:30.242Z,54059267.0,classification_count
3,1449,308319833,AfuoO.,2233913.0,e6058d710b12e4416d16,17178,Draw rectangles around the gemma cups,5.8,NaN,NaN,...,Draw a rectangle around every visible gemma cup.,"[{'x': 845.0218505859375, 'y': 953.13122558593...",75025965.0,17178.0,19.0,2020-12-21T08:04:06.830Z,2021-02-09T21:25:03.363Z,2021-02-09T21:25:03.350Z,54059274.0,classification_count


In [7]:
df_xµd["annotations"][0]

'[{"task":"T0","task_label":"Draw a rectangle around every visible gemma cup.","value":[{"x":696.6875610351562,"y":667.3584594726562,"tool":0,"frame":0,"width":60.6689453125,"height":69.86126708984375,"details":[],"tool_label":"Draw rectangle around the cups"},{"x":654.4031372070312,"y":788.6964111328125,"tool":0,"frame":0,"width":33.0921630859375,"height":36.76904296875,"details":[],"tool_label":"Draw rectangle around the cups"},{"x":858.471435546875,"y":779.504150390625,"tool":0,"frame":0,"width":58.83050537109375,"height":47.79974365234375,"details":[],"tool_label":"Draw rectangle around the cups"},{"x":786.771728515625,"y":886.1343994140625,"tool":0,"frame":0,"width":56.9920654296875,"height":40.44598388671875,"details":[],"tool_label":"Draw rectangle around the cups"}]}]'

### Fix user_agent

In [8]:
dfx.user_agent = dfx.user_agent.str[13:28]
dfx.user_agent.unique()

array(['Windows NT 10.0', 'X11; Linux x86_', 'Linux; Android ',
       'iPhone; CPU iPh', 'Macintosh; Inte', 'Windows NT 6.3;',
       'X11; CrOS x86_6', 'Windows NT 6.1;', 'iPad; CPU OS 12'],
      dtype=object)

In [9]:
dfx.user_agent = np.select(
    [
        dfx.user_agent == "Windows NT 10.0",
        dfx.user_agent == "Macintosh; Inte",
        dfx.user_agent == "Linux; Android ",
        dfx.user_agent == "X11; CrOS x86_6",
        dfx.user_agent == "Windows NT 6.1;",
        dfx.user_agent == "iPhone; CPU iPh",
        dfx.user_agent == "X11; Linux x86_",
        dfx.user_agent == "Windows NT 6.3;",
        dfx.user_agent == "iPad; CPU OS 12",
    ],
    [
        "Windows 10",
        "OSX",
        "Android",
        "Chrome OS",
        "Windows 7",
        "iPhone",
        "Linux",
        "Windows 8.1",
        "iPad",
    ],
    default="Oops",
)
dfx.user_agent.unique()

array(['Windows 10', 'Linux', 'Android', 'iPhone', 'OSX', 'Windows 8.1',
       'Chrome OS', 'Windows 7', 'iPad'], dtype=object)

### Keep only needed columns

In [10]:
dfx.shape

(2118, 49)

In [11]:
dfx.columns

Index(['index', 'classification_id', 'user_name', 'user_id', 'user_ip',
       'workflow_id', 'workflow_name', 'workflow_version', 'gold_standard',
       'expert', 'subject_ids', 'width', 'height', 'opt_in', 'messageshown',
       'selected_at', 'already_seen', 'selection_state', 'finished_workflow',
       'user_has_finished_workflow', 'source', 'session', 'started_at',
       'user_agent', 'utc_offset', 'finished_at', 'live_project',
       'user_language', 'user_group_ids', 'workflow_translation_id',
       'seen_before', 'clientwidth', 'clientheight', 'naturalwidth',
       'naturalheight', 'retired_bool', 'creation_time', 'filename', 'task',
       'task_label', 'value', 'id', 'workflow_id', 'classifications_count',
       'created_at', 'updated_at', 'retired_at', 'subject_id',
       'retirement_reason'],
      dtype='object')

In [12]:
df_keep = dfx[
    [
        "user_name",
        "user_agent",
        "filename",
        "value",
        "classifications_count",
        "creation_time",
    ]
]
df_keep.head(3)

,user_name,user_agent,filename,value,classifications_count,creation_time
0,348850140,Windows 10,b0t0i7Up-N7SKVIpgIHmL27YfD1M.jpg,"[{'x': 696.6875610351562, 'y': 667.35845947265...",19.0,2021-02-09 22:48:36 UTC
4,AfuoO.,Linux,b0K9go-7zp6ItpVeN-bmIwAgU12Y.jpg,[],29.0,2021-02-09 21:50:43 UTC
3,AfuoO.,Linux,b0LLoYT8lOeGd8hpo6aLHmL-5pT4.jpg,"[{'x': 845.0218505859375, 'y': 953.13122558593...",19.0,2021-02-09 21:49:36 UTC


### Count observations

In [13]:
df_keep["rect_count"] = df_keep.value.apply(lambda x: len(x))
df_keep.head()
# df_keep.value.apply(lambda x: len(x))

<ipython-input-13-22a59b67e13d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_keep["rect_count"] = df_keep.value.apply(lambda x: len(x))


,user_name,user_agent,filename,value,classifications_count,creation_time,rect_count
0,348850140,Windows 10,b0t0i7Up-N7SKVIpgIHmL27YfD1M.jpg,"[{'x': 696.6875610351562, 'y': 667.35845947265...",19.0,2021-02-09 22:48:36 UTC,4
4,AfuoO.,Linux,b0K9go-7zp6ItpVeN-bmIwAgU12Y.jpg,[],29.0,2021-02-09 21:50:43 UTC,0
3,AfuoO.,Linux,b0LLoYT8lOeGd8hpo6aLHmL-5pT4.jpg,"[{'x': 845.0218505859375, 'y': 953.13122558593...",19.0,2021-02-09 21:49:36 UTC,12
7,AfuoO.,Linux,b0XI7z-ehGUnU0CBfclWm8xQ8508.jpg,[],24.0,2021-02-09 21:55:19 UTC,0
5,AfuoO.,Linux,b0dZAQThtozJxxsMRo5ewY2ebGss.jpg,[],22.0,2021-02-09 21:51:26 UTC,0


### Tidy up

In [14]:
df_keep.columns

Index(['user_name', 'user_agent', 'filename', 'value', 'classifications_count',
       'creation_time', 'rect_count'],
      dtype='object')

In [15]:
tidy = (
    pd.concat([df_keep.drop(["value"], axis=1), df_keep.value.apply(pd.Series)], axis=1)
    .melt(
        id_vars=[
            "user_name",
            "filename",
            "user_agent",
            "rect_count",
            "classifications_count",
            "creation_time",
        ],
        var_name="dummy",
        value_name="rectangle",
    )
    .drop(["dummy"], axis=1)
    .dropna(subset=["user_name", "filename", "user_agent", "rect_count"])
)

tidy = pd.concat(
    [tidy.drop(["rectangle"], axis=1), tidy.rectangle.apply(pd.Series)], axis=1
)[
    [
        "user_name",
        "filename",
        "creation_time",
        "user_agent",
        "rect_count",
        "classifications_count",
        "x",
        "y",
        "width",
        "height",
    ]
]

tidy = (tidy >> filter(_.classifications_count >= 0)).reset_index().drop_duplicates()
tidy

,index,user_name,filename,creation_time,user_agent,rect_count,classifications_count,x,y,width,height
0,0,348850140,b0t0i7Up-N7SKVIpgIHmL27YfD1M.jpg,2021-02-09 22:48:36 UTC,Windows 10,4,19.0,696.687561,667.358459,60.668945,69.861267
1,1,AfuoO.,b0K9go-7zp6ItpVeN-bmIwAgU12Y.jpg,2021-02-09 21:50:43 UTC,Linux,0,29.0,NaN,NaN,NaN,NaN
2,2,AfuoO.,b0LLoYT8lOeGd8hpo6aLHmL-5pT4.jpg,2021-02-09 21:49:36 UTC,Linux,12,19.0,845.021851,953.131226,92.998230,103.579956
3,3,AfuoO.,b0XI7z-ehGUnU0CBfclWm8xQ8508.jpg,2021-02-09 21:55:19 UTC,Linux,0,24.0,NaN,NaN,NaN,NaN
4,4,AfuoO.,b0dZAQThtozJxxsMRo5ewY2ebGss.jpg,2021-02-09 21:51:26 UTC,Linux,0,22.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
39319,61414,singtolive,b0rpcXs6lAtSwkF4uxgoWTfRGU.jpg,2021-02-09 21:59:27 UTC,OSX,0,16.0,NaN,NaN,NaN,NaN
39320,61415,singtolive,b0tXpblaTKCgX1hQTGjcp6QT182E.jpg,2021-02-09 21:58:58 UTC,OSX,1,26.0,NaN,NaN,NaN,NaN
39321,61416,singtolive,b0uPIHx8926mmzez-pMIONV-2EdE.jpg,2021-02-09 21:59:59 UTC,OSX,0,21.0,NaN,NaN,NaN,NaN
39322,61417,tat_qa,b0DaRvfMv4Ve9fQ7-vmzY5lXanY.jpg,2021-02-10 21:32:09 UTC,Windows 10,0,23.0,NaN,NaN,NaN,NaN


#  Explore the data

In [16]:
tidy[
    (tidy.user_name == "Brooker1957")
    & (tidy.filename == "b0xhA8TCuQtLRbirX369iE7dJvUE.jpg")
].dropna().reset_index().drop(["index"], axis=1).drop_duplicates().sort_values(
    ["user_name", "filename", "x", "y", "width", "height"]
).dropna()

,level_0,user_name,filename,creation_time,user_agent,rect_count,classifications_count,x,y,width,height
35,7096,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2021-02-04 17:27:52 UTC,Windows 10,15,27.0,525.379517,615.052002,79.688782,47.813232
36,8447,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2020-12-22 17:54:19 UTC,Windows 10,15,27.0,539.868469,632.438660,73.893127,33.324341
87,19298,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2021-02-03 17:42:31 UTC,Windows 10,15,27.0,542.766174,626.643066,52.159851,40.568848
68,15229,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2021-01-15 17:32:21 UTC,Windows 10,15,27.0,545.663940,623.745300,62.302063,55.057739
61,13872,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2020-12-22 18:01:09 UTC,Windows 10,15,27.0,552.908325,616.500916,60.853149,66.648682
...,...,...,...,...,...,...,...,...,...,...,...
1,312,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2020-12-22 18:01:09 UTC,Windows 10,15,27.0,977.431885,807.753845,99.973145,133.297485
89,19300,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2021-02-04 17:27:52 UTC,Windows 10,15,27.0,978.880798,819.344910,68.097595,86.933167
19,4380,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2020-12-22 18:01:09 UTC,Windows 10,15,27.0,980.329712,590.421021,92.728638,95.626465
15,3026,Brooker1957,b0xhA8TCuQtLRbirX369iE7dJvUE.jpg,2021-02-03 17:42:31 UTC,Windows 10,15,27.0,980.329712,836.731567,79.688599,68.097534


In [17]:
no_rect_df = tidy[
    ["user_name", "filename", "creation_time", "user_agent", "rect_count"]
].drop_duplicates()
df_stats = (
    no_rect_df.groupby("filename")
    .rect_count.agg(
        count="count",
        min="min",
        max="max",
        mean="mean",
        median="median",
        std="std",
        mode=lambda x: x.mode(),
    )
    .reset_index()
    .sort_values("filename")
)
df_stats

,filename,count,min,max,mean,median,std,mode
0,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,25,1,13,8.880000,9,2.386071,9
1,b00s9swai4uy9RmDXcnkiY1PpZ2M.jpg,28,1,5,3.928571,4,0.899735,4
2,b0BBwUoWkQfXpAUn8gvPh0AJy1hQ.jpg,30,0,1,0.100000,0,0.305129,0
3,b0DaRvfMv4Ve9fQ7-vmzY5lXanY.jpg,28,0,1,0.107143,0,0.314970,0
4,b0Dkf149PgiY8uzDB1naeNyC0L9A.jpg,16,0,5,3.125000,4,1.746425,4
5,b0F36HFy2D01M3WdyopeVzzKaP4.jpg,22,0,4,0.227273,0,0.869144,0
6,b0G-uQyqygMNBuOC3sBij5u5NruI.jpg,24,0,3,0.916667,1,0.775532,1
7,b0K9go-7zp6ItpVeN-bmIwAgU12Y.jpg,34,0,4,0.235294,0,0.780785,0
8,b0KXwBrE57rCtnxjL2jKk0AXGwCI.jpg,22,0,14,8.500000,10,3.961121,10
9,b0Kn3PC0uM7Bezjtf3JSNN4utEFQ.jpg,33,0,4,0.272727,0,0.801277,0


In [18]:
observations = ["Select an observation"] + sorted(df_stats.filename.unique().tolist())
obs_selected = widgets.Dropdown(
    options=observations,
    description="Select an observation:",
)

user_selected = widgets.Dropdown(
    options=[],
    description="Select a user:",
)

date_selected = widgets.Dropdown(
    options=[],
    description="Select a date:",
)

shape_selected = widgets.Dropdown(
    options=["Rectangle", "Circle"],
    description="Draw shape:",
    value="Rectangle",
)

button = widgets.Button(description="Render")

image_with_rects = widgets.Output(layout={"border": "1px solid black"})
text_rects = widgets.Output(layout={"border": "1px solid black"})


dataframe: widgets.Output = widgets.Output(layout={"border": "1px solid black"})
stats_output = widgets.Output(layout={"border": "1px solid black"})


def update_overview(observation, user, date_, shape, update_user, update_date):
    dataframe.clear_output()
    with dataframe:
        display(
            no_rect_df.drop("filename", axis=1)[
                no_rect_df.filename == observation
            ].reset_index()
        )

    stats_output.clear_output()
    with stats_output:
        display(df_stats[df_stats.filename == observation].reset_index())

    if update_user == "update":
        user_selected.options = ["None", "All"] + sorted(
            tidy[tidy.filename == observation].user_name.unique().tolist()
        )
    elif update_user == "clear":
        user_selected.options = []
    if update_date == "update":
        date_selected.options = ["All"] + sorted(
            tidy[(tidy.filename == observation) & (tidy.user_name == user)]
            .creation_time.unique()
            .tolist()
        )
    elif update_date == "clear":
        date_selected.options = []

#     if observation == "Select an observation":
#         return
#     elif user == "None":
#         rects = None
#     elif user == "All":
    rects = tidy[(tidy.filename == observation)]
#     else:
#         rects = tidy[(tidy.filename == observation) & (tidy.user_name == user)]

#     if rects is None:
#         pass
#     elif date_ == "None":
#         rects = None
#     elif date_ != "All":
#         rects = rects[rects.creation_time == date_]

    if rects is not None:
        rects = rects.reset_index().dropna()[["x", "y", "width", "height"]]
    image_with_rects.clear_output()
    with image_with_rects:
        img = PilImage.open(f"../data_in/images/{obs_selected.value}")
        fig, ax = plt.subplots()
        fig.set_size_inches(14, 14)
        ax.set_axis_off()
        ax.imshow(img)
        if rects is not None:
            rects = rects.reset_index().dropna()[["x", "y", "width", "height"]]
            colors = [
                random.choice(list(mcolors.CSS4_COLORS.keys()))
                for _ in range(rects.shape[0])
            ]
            for x, y, w, h, c in zip(
                rects.x, rects.y, rects.width, rects.height, colors
            ):
                if shape == "Circle":
                    ax.add_patch(
                        patches.Circle(
                            (x + w // 2, y + h // 2),
                            8,
                            linewidth=8,
                            edgecolor=c,
                            facecolor="none",
                        )
                    )
                elif shape == "Rectangle":
                    ax.add_patch(
                        patches.Rectangle(
                            (x, y), w, h, linewidth=2, edgecolor=c, facecolor="none"
                        )
                    )
        plt.show()

    text_rects.clear_output()
    with text_rects:
        if rects is not None:
            display(rects)


def on_observation_selected(change):
    update_overview(
        observation=change.new,
        user=user_selected.value,
        date_=date_selected.value,
        shape=shape_selected.value,
        update_user="update",
        update_date="clear",
    )


def on_user_selected(change):
    update_overview(
        observation=obs_selected.value,
        user=change.new,
        date_=date_selected.value,
        shape=shape_selected.value,
        update_user="",
        update_date="update",
    )


def on_timestamp_selected(change):
    update_overview(
        observation=obs_selected.value,
        user=user_selected.value,
        date_=date_selected.value,
        shape=shape_selected.value,
        update_user="",
        update_date="",
    )


def on_shape_selected(change):
    update_overview(
        observation=obs_selected.value,
        user=user_selected.value,
        date_=date_selected.value,
        shape=change.new,
        update_user="",
        update_date="",
    )


obs_selected.observe(on_observation_selected, names="value")
# user_selected.observe(on_user_selected, names="value")
# date_selected.observe(on_timestamp_selected, names="value")
# shape_selected.observe(on_shape_selected, names="value")
# button.on_click(on_button_clicked)

display(
    HBox([obs_selected, user_selected, date_selected, shape_selected, button]),
    stats_output,
    HBox([dataframe, image_with_rects, text_rects]),
)

Output(layout=Layout(border='1px solid black'))

In [19]:
images_list = ["Select an observation"] + sorted(df_stats.filename.unique().tolist())
dd_image = widgets.Dropdown(
    options=images_list,
    description="Select an observation:",
)

is_print_all = widgets.Checkbox(
    value=False,
    description="Print all annotations centers",
    disabled=False,
    indent=False,
)
is_print_centers = widgets.Checkbox(
    value=False, description="Print all kmeans centers", disabled=False, indent=False
)
is_print_rectangles = widgets.Checkbox(
    value=True, description="Print rectangles", disabled=False, indent=False
)

image_output = widgets.Output(layout={"border": "1px solid black"})
image_stat_output = widgets.Output(layout={"border": "1px solid black"})
user_gt = widgets.Output(layout={"border": "1px solid black"})
value_counts = widgets.Output(layout={"border": "1px solid black"})


def print_ground_truth(
    observation: str,
    print_all: bool = False,
    print_centers: bool = False,
    print_rectangles: bool = True,
):
    image_stat_output.clear_output()
    with image_stat_output:
        display(df_stats[df_stats.filename == observation].reset_index())

    vc = no_rect_df[no_rect_df.filename == observation]["rect_count"].value_counts()
    if len(vc) > 1:
        v = vc.index.to_list()[:2]
        c = vc.to_list()[:2]
        if c[0] > 2 * c[1]:
            allowed_counts = [v[0]]
        else:
            allowed_counts = v
    else:
        allowed_counts = vc.index.to_list()

    rects = tidy[
        (tidy.filename == observation) & (tidy.rect_count.isin(allowed_counts))
    ].dropna()[["x", "y", "width", "height"]]
    rects = rects[
        (rects.width < 2 * rects.width.median())
        & (rects.height < 2 * rects.height.median())
    ].reset_index()
    image_output.clear_output()
    with image_output:
        img = PilImage.open(f"../data_in/images/{observation}")
        fig, ax = plt.subplots()
        fig.set_size_inches(14, 14)
        ax.set_axis_off()
        ax.imshow(img)
        if (rects is not None) and (rects.shape[0] > 0):
            rects = rects.assign(
                cx=rects.x + rects.width // 2,
                cy=rects.y + rects.height // 2,
            )
            X = [(cx, cy) for cx, cy in zip(rects.cx, rects.cy)]
            kmeans = KMeans(
                n_clusters=max(allowed_counts),
                random_state=42,
            ).fit(X)
            y_pred = kmeans.predict(X)
            if print_all:
                ax.scatter(rects.cx, rects.cy, c=y_pred, alpha=0.5)
            if print_rectangles:
                rects["y_pred"] = y_pred
                rects = rects.groupby("y_pred").median().reset_index()
                for x, y, w, h in zip(rects.x, rects.y, rects.width, rects.height):
                    ax.add_patch(
                        patches.Rectangle(
                            (x, y), w, h, linewidth=2, edgecolor="r", facecolor="none"
                        )
                    )
            if print_centers:
                centers = kmeans.cluster_centers_
                ax.scatter(centers[:, 0], centers[:, 1], c="b", s=200, alpha=0.5)
        plt.show()

    user_gt.clear_output()
    with user_gt:
        display(
            no_rect_df[no_rect_df.filename == observation]
            .reset_index()
            .drop(["filename", "user_agent"], axis=1)
        )

    value_counts.clear_output()
    with value_counts:
        display(
            no_rect_df[no_rect_df.filename == observation]["rect_count"].value_counts()
        )


def on_image_selected(change):
    print_ground_truth(
        observation=change.new,
        print_all=is_print_all.value,
        print_centers=is_print_centers.value,
        print_rectangles=is_print_rectangles.value,
    )


def on_print_all_changed(change):
    print_ground_truth(
        observation=dd_image.value,
        print_all=change.new,
        print_centers=is_print_centers.value,
        print_rectangles=is_print_rectangles.value,
    )


def on_print_centers_changed(change):
    print_ground_truth(
        observation=dd_image.value,
        print_all=is_print_all.value,
        print_centers=change.new,
        print_rectangles=is_print_rectangles.value,
    )


def on_print_rectangles_changed(change):
    print_ground_truth(
        observation=dd_image.value,
        print_all=is_print_all.value,
        print_centers=is_print_centers.value,
        print_rectangles=change.new,
    )


dd_image.observe(on_image_selected, names="value")

is_print_all.observe(on_print_all_changed, names="value")
is_print_centers.observe(on_print_centers_changed, names="value")
is_print_rectangles.observe(on_print_rectangles_changed, names="value")

display(
    HBox([dd_image, is_print_all, is_print_centers, is_print_rectangles]),
    image_stat_output,
    HBox([image_output, user_gt, value_counts]),
)

Output(layout=Layout(border='1px solid black'))

In [20]:
df_lst = []
for filename in df_stats.filename.unique().tolist():
    vc = no_rect_df[no_rect_df.filename == filename]["rect_count"].value_counts()
    if len(vc) > 1:
        v = vc.index.to_list()[:2]
        c = vc.to_list()[:2]
        if c[0] > 2 * c[1]:
            allowed_counts = [v[0]]
        else:
            allowed_counts = v
    else:
        allowed_counts = vc.index.to_list()

    rects = (
        tidy[(tidy.filename == filename) & (tidy.rect_count.isin(allowed_counts))]
        .drop(
            [
                "index",
                "user_name",
                "creation_time",
                "user_agent",
                "rect_count",
                "classifications_count",
            ],
            axis=1,
        )
        .dropna()
    )
    rects = rects[
        (rects.width < 2 * rects.width.median())
        & (rects.height < 2 * rects.height.median())
    ].reset_index()
    if (rects is not None) and (rects.shape[0] > 0):
        X = [
            (cx, cy)
            for cx, cy in zip(rects.x + rects.width // 2, rects.y + rects.height // 2)
        ]
        rects["y_pred"] = (
            KMeans(
                n_clusters=max(allowed_counts),
                random_state=42,
            )
            .fit(X)
            .predict(X)
        )
        df_lst.append(
            rects.assign(
                x=rects.groupby("y_pred", dropna=True,)[
                    "x"
                ].transform("median"),
                y=rects.groupby("y_pred", dropna=True,)[
                    "y"
                ].transform("median"),
                width=rects.groupby("y_pred", dropna=True,)[
                    "width"
                ].transform("median"),
                height=rects.groupby("y_pred", dropna=True,)[
                    "height"
                ].transform("median"),
            )
            .reset_index(drop=True)
            .drop(["y_pred", "index"], axis=1)
            .drop_duplicates()
            .reset_index(drop=True)
        )
    else:
        df_lst.append(
            pd.DataFrame(
                [[filename, np.NaN, np.NaN, np.NaN, np.NaN]],
                columns=["filename", "x", "y", "width", "height"],
            )
        )


df_final = pd.concat(df_lst)
# df_final.to_csv(
#     path_or_buf="../../output/zooniverse/boxes.csv",
#     index=False,
# )
df_final

,filename,x,y,width,height
0,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,690.798218,560.637146,45.351654,39.815979
1,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,1004.213104,791.837433,43.049530,46.096863
2,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,611.747009,738.196106,91.825195,84.914490
3,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,905.668762,749.575195,79.746094,84.656433
4,b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg,708.588226,613.453003,94.325226,72.136810
...,...,...,...,...,...
1,b0y2tMS9MhLdCvc9aRveQaHxqsA.jpg,601.347351,637.444794,68.529083,62.340851
2,b0y2tMS9MhLdCvc9aRveQaHxqsA.jpg,804.126221,773.024658,50.010712,39.594574
3,b0y2tMS9MhLdCvc9aRveQaHxqsA.jpg,707.135193,845.243683,57.955414,50.693451
4,b0y2tMS9MhLdCvc9aRveQaHxqsA.jpg,806.418121,705.034851,54.851562,47.799500


In [21]:
dd_final_image = widgets.Dropdown(options=sorted(df_final.filename.unique().tolist()))

final_image_output = widgets.Output(layout={"border": "1px solid black"})
rects_output = widgets.Output(layout={"border": "1px solid black"})


def print_final_rects(change):
    final_image_output.clear_output()
    rects = df_final[df_final.filename == change.new]
    with final_image_output:
        img = PilImage.open(f"../data_in/images/{change.new}")
        fig, ax = plt.subplots()
        fig.set_size_inches(14, 14)
        ax.set_axis_off()
        ax.imshow(img)
        if (rects is not None) and (rects.shape[0] > 0):
            for x, y, w, h in zip(rects.x, rects.y, rects.width, rects.height):
                ax.add_patch(
                    patches.Rectangle(
                        (x, y),
                        w,
                        h,
                        linewidth=2,
                        edgecolor="r",
                        facecolor="none",
                    )
                )
        plt.show()
    rects_output.clear_output()
    with rects_output:
        display(rects)


dd_final_image.observe(print_final_rects, names="value")
display(dd_final_image, HBox([final_image_output, rects_output]))

Dropdown(options=('b00mXpDa2KjZ1b8xWs1aq7vRVvI.jpg', 'b00s9swai4uy9RmDXcnkiY1PpZ2M.jpg', 'b0BBwUoWkQfXpAUn8gvP…